In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score

In [2]:
# we are implementing a knn-classifier with L_p distance
# this is our first implementation which builds a distance matrix
# it becomes very inefficient when the dimension of dataset is big

def distanceLp(v1, v2, p):
    return np.power(np.absolute(np.sum((v1 - v2)) ** p), (1./p))

def distanceL2(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))

def distMatrix(train_X, test_X):
    # we build a matrix of size (len(test_X), len(train_X))
    # such that each row i is the distances from test[i] to all training points 
    dist_matrix = np.zeros((test_X.shape[0], train_X.shape[0]))
    for i in range(test_X.shape[0]):
        for j in range(train_X.shape[0]):
            #print test_X[i], train_X[j]
            if type(test_X)== type(np.array([])):
                dist_matrix[i][j] = distanceL2(test_X[i], train_X[j])
            else:
                dist_matrix[i][j] = distanceL2(test_X[i].toarray(), train_X[j].toarray())
    return  dist_matrix

def k_nearest_neighbors(dist_matrix,k):
    # this function builds a matrix of size (len(dist_matrix), k)
    # for each test[i], the ith row in the matrix represents the index of k nearest neighbors
    dist_matrix_largest_k = np.zeros((len(dist_matrix), k))
    #print dist_matrix_largest_k 
    for i in range(len(dist_matrix)):
        #print 'i=',i 
        dist_matrix_largest_k[i] = np.argpartition(dist_matrix[i],k)[:k] 
        #print np.argpartition(dist_matrix[i],k)[:k] 
        #print dist_matrix_largest_k[i]
    return dist_matrix_largest_k.astype('int')
    
def find_majority(votes):
    vote_count = Counter(votes)
    top_one = vote_count.most_common(1)
    return top_one[0][0]

def knn_classifier(X_train, X_test, Y_train, k):
    dist_M = distMatrix(X_train, X_test)
    print 'construct distance matrix done'
    dist_M_k =  k_nearest_neighbors(dist_M, k)
    print 'find k nearest neubours matrix done'
    results = []
    for i in range(X_test.shape[0]):
        k_index = dist_M_k[i]
        results.append(find_majority(Y_train[k_index])) 
    return results

In [3]:
# use iris dataset for algorithm design test
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data # we only take the first two features.
Y = iris.target

m=0.8*len(X)
X_train = X[:m]
X_test = X[m:]
Y_train = Y[:m]
Y_test = Y[m:]

## test our implementation 
pred = knn_classifier(X_train, X_test, Y_train,5)
print accuracy_score(pred, Y_test)

construct distance matrix done
find k nearest neubours matrix done
0.8


/Users/yuedong/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/yuedong/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/yuedong/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/yuedong/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [177]:
import pandas as pd
import csv 
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import HTMLParser
from nltk import word_tokenize
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import pickle
from sklearn import svm, linear_model, naive_bayes 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

X = pd.read_csv("X_train.csv")['0']
final_test = pd.read_csv("X_test.csv")['0']
Y = pd.read_csv("Y_train.csv")['0']
print 'read X, Y, final_test done'
#type(X)

from collections import Counter
Counter(Y)

read X, Y, final_test done


Counter({'hockey': 13994,
         'movies': 14847,
         'nba': 12325,
         'news': 13986,
         'nfl': 13392,
         'politics': 13205,
         'soccer': 14224,
         'worldnews': 14027})

In [178]:
def tfidf_vec(X_train,X_test,final_test):
    print ("Using Tfidf Vectorizer")
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_features=15000)
    #
    train_data_features = vectorizer.fit_transform(X_train)
    
    X_test_tfidf = vectorizer.transform(X_test)
    
    final_test_tfidf = vectorizer.transform(final_test)
    
    #train_data_features = train_data_features.toarray()
    #X_test_tfidf = X_test_tfidf.toarray()
    #final_test_tfidf = final_test_tfidf.toarray()
    return train_data_features,X_test_tfidf, final_test_tfidf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
X_train_tfidf, X_test_tfidf, test_tfidf= tfidf_vec(X_train, X_test, final_test)

Using Tfidf Vectorizer


In [ ]:
import time

start_time = time.time()

pred = knn_classifier(X_train_tfidf[1:1000], X_test_tfidf[1:1000], y_train[1:1000].as_matrix(),3)

print 'run-time:', time.time()- start_time 

print 'overall accuracy', accuracy_score(y_test[1:1000].as_matrix(), pred)
#print(classification_report(y_test[1:1000], pred))  

In [ ]:
def tfidf_vec_for_testing(X_train,final_test):
    print ("Using Tfidf Vectorizer")
    vectorizer = TfidfVectorizer(max_features=500)

    train_data_features = vectorizer.fit_transform(X_train)
    final_test_tfidf = vectorizer.transform(final_test)
    
    return train_data_features, final_test_tfidf

X_train_tfidf, test_tfidf= tfidf_vec_for_testing(X, final_test)
pred_final = knn(X_train_tfidf, test_tfidf, y_test,5)
print 'overall accuracy', accuracy_score(y_test, predicted)
print(classification_report(y_test, predicted))  

In [ ]:
# a more efficient way to implement knn without using building distance matrix
# we are implementing a knn-classifier with the euclidean distance

def distanceLp(v1, v2, p):
    return np.power(np.sum((np.absolute(v1 - v2)) ** p), (1./p))
    print 
def distanceL2(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))

def distVector(train_X, test_X_one_instance, p):
    # a vector of distances from a test point to all train points

    dist_vector = np.zeros(train_X.shape[0])
    
    for i in range(train_X.shape[0]):
        
        if type(test_X_one_instance)== type(np.array([])):
            # vectors are numpy array
            dist_vector[i] = distanceLp(test_X_one_instance, train_X[i], p)
            
        else:
            # vectors are sparse matrix
            dist_vector[i] = distanceLp(test_X_one_instance.toarray(), train_X[i].toarray(), p)
    #print dist_vector
    return  dist_vector

def k_nearest_neighbors_v(distVector,k):
    
    dist_vector_largest_k = np.argpartition(distVector,k)[:k] 
        
    return  dist_vector_largest_k.astype('int')
    
def find_majority(votes):
    vote_count = Counter(votes)
    top_one = vote_count.most_common(1)
    return top_one[0][0]

def knn_classifier_v(X_train, X_test, Y_train, k, p):
    
    results = []
    for i in range(X_test.shape[0]):
        dist_v = distVector(X_train, X_test[i], p)
        
        dist_v_k_index = k_nearest_neighbors_v(dist_v, k)
        
        results.append(find_majority(Y_train.as_matrix()[dist_v_k_index])) 
    return results

In [ ]:
start_time = time.time()

pred = knn_classifier_v(X_train_tfidf[1:100], X_test_tfidf[1:100], y_train[1:100],20, 2)

print 'run-time:', time.time()- start_time 
print 'overall accuracy', accuracy_score(y_test[1:100], pred)
#print(classification_report(y_test[1:1000], pred))  

In [ ]:
for i in [1,2,3,5,10,15,20]:
    for j in [1,2,3,5]:
        pred = knn_classifier_v(X_train_tfidf[1:10000], X_test_tfidf[1:10000], y_train[1:10000],i,j)
        print 'k=',i,'p=',j, 'overall accuracy', accuracy_score(y_test[1:10000], pred)